In [22]:
import torch 
import pandas
from torch_geometric.datasets import AMiner 
import os
from torch.utils.data import DataLoader  

In [ ]:

# TODO: load data set and split into batches in order to run in my computer 
# TODO: define the mata-paths 
# TODO: Do meta-path random walks and save this as a vector
# TODO: From the vectors generated plug into skip-gram model and do negative sampling 

In [2]:
cwd = os.getcwd()
save_path_for_data = os.path.join(cwd, 'Aminer')
print(save_path_for_data)

c:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\metapath2vec\Aminer


In [3]:
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu')
dataset = AMiner(root= save_path_for_data)

In [4]:
data = dataset[0]
print(data)

HeteroData(
  author={
    y=[246678],
    y_index=[246678],
    num_nodes=1693531,
  },
  venue={
    y=[134],
    y_index=[134],
    num_nodes=3883,
  },
  paper={ num_nodes=3194405 },
  (paper, written_by, author)={ edge_index=[2, 9323605] },
  (author, writes, paper)={ edge_index=[2, 9323605] },
  (paper, published_in, venue)={ edge_index=[2, 3194405] },
  (venue, publishes, paper)={ edge_index=[2, 3194405] }
)


In [5]:
print(data.edge_index_dict[('paper', 'published_in', 'venue')])
print(data.edge_index_dict[('paper', 'written_by', 'author')])
print(data.edge_index_dict[('author', 'writes', 'paper')])
print(data.edge_index_dict[('venue', 'publishes', 'paper')])

tensor([[      0,       1,       2,  ..., 3194402, 3194403, 3194404],
        [   2190,    2190,    2190,  ...,    3148,    3148,    3148]])
tensor([[      0,       1,       2,  ..., 3194404, 3194404, 3194404],
        [      0,       1,       2,  ...,    4393,   21681,  317436]])
tensor([[      0,       1,       2,  ...,    4393,   21681,  317436],
        [      0,       1,       2,  ..., 3194404, 3194404, 3194404]])
tensor([[   2190,    2190,    2190,  ...,    3148,    3148,    3148],
        [      0,       1,       2,  ..., 3194402, 3194403, 3194404]])


In [6]:
print(data.edge_index_dict)

{('paper', 'written_by', 'author'): tensor([[      0,       1,       2,  ..., 3194404, 3194404, 3194404],
        [      0,       1,       2,  ...,    4393,   21681,  317436]]), ('author', 'writes', 'paper'): tensor([[      0,       1,       2,  ...,    4393,   21681,  317436],
        [      0,       1,       2,  ..., 3194404, 3194404, 3194404]]), ('paper', 'published_in', 'venue'): tensor([[      0,       1,       2,  ..., 3194402, 3194403, 3194404],
        [   2190,    2190,    2190,  ...,    3148,    3148,    3148]]), ('venue', 'publishes', 'paper'): tensor([[   2190,    2190,    2190,  ...,    3148,    3148,    3148],
        [      0,       1,       2,  ..., 3194402, 3194403, 3194404]])}


In [32]:
print(data['author'].num_nodes)

1693531


In [7]:
metapath = [
    ('paper', 'written_by', 'author'),
    ('author', 'writes', 'paper'),
    ('paper', 'published_in', 'venue'),
    ('venue', 'publishes', 'paper')
]

In [24]:
from torch_geometric.nn import MetaPath2Vec

model = MetaPath2Vec(data.edge_index_dict, embedding_dim=128,
                     metapath=metapath, walk_length=50, context_size=7,
                     walks_per_node=5, num_negative_samples=5,
                     sparse=True).to(device)

In [38]:
loader = model.loader(batch_size=128, shuffle=True, num_workers=6) # decreased batch size in order to run
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(loader))


In [39]:
def train(epoch, log_steps=100, eval_steps=2000):
    
    model.train()

    total_loss = 0
    for i, (pos_rw, neg_rw) in enumerate(loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        if (i + 1) % log_steps == 0:
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Loss: {total_loss / log_steps:.4f}'))
            total_loss = 0

        if (i + 1) % eval_steps == 0:
            acc = test()
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Acc: {acc:.4f}'))


@torch.no_grad()
def test(train_ratio=0.1):
    model.eval()

    z = model('author', batch=data['author'].y_index.to(device))
    y = data['author'].y

    perm = torch.randperm(z.size(0))
    train_perm = perm[:int(z.size(0) * train_ratio)]
    test_perm = perm[int(z.size(0) * train_ratio):]

    return model.test(z[train_perm], y[train_perm], z[test_perm], y[test_perm],
                      max_iter=150)

In [40]:
for epoch in range(1):
    train(epoch)
    acc = test()
    print(f'Epoch: {epoch}, Accuracy: {acc:.4f}')

KeyboardInterrupt: 